In [ ]:
# Enable interactive plots with backend 'notebook'
%matplotlib notebook

# Enable connecting with ipython console --existing
# %connect_info

# print code version (hash of checked out version)
!git log -1

# print date and time of script execution
import datetime
print("\nNotebook executed at at {} in following directory:".format(datetime.datetime.now()))
%cd ..
%pwd

## Imports

In [ ]:
from optimize.bpop_optimize_stn import *

from matplotlib import pyplot as plt

def plot_responses(responses):
    """ Quick plot function for responses dict"""
    for resp_name, traces in responses.iteritems():
        plt.figure()
        plt.plot(traces['time'], traces['voltage'])
        plt.suptitle(resp_name)

    plt.show(block=False)

## Parameters & Cell Model

In [ ]:
# Make parameters
gleak_orig = 7.84112e-05
gleak_fit = 12.43169e-5 # fit to match Zin_DC (see praxis_passive.py)

dend_gl_param = ephys.parameters.NrnSectionParameter(
                    name		= 'gleak_dend_param',
                    param_name	= gleak_name,
                    locations	= [dendritic_region],
                    bounds		= [gleak_fit, gleak_fit],
                    value		= gleak_fit,
                    frozen		= True)

cm_orig = 1.0
cm_fit = cm_orig * (gleak_fit / gleak_orig) # preserve membrane time constant

dend_cm_param = ephys.parameters.NrnSectionParameter(
                    name		= 'cm_dend_param',
                    param_name	= 'cm',
                    locations	= [dendritic_region],
                    bounds		= [cm_fit, cm_fit],
                    value		= cm_fit,
                    frozen		= True)

# Frozen parameters are passive parameters fit previously in passive model
dend_passive_params = [dend_gl_param, dend_cm_param]

# Free parameters are active conductances with large impact on response
dend_active_params = dend_gbar_params # Free parameters
dend_all_params = dend_passive_params + dend_active_params

## Features & Objectives

### Test Protocol (not required)

Run cells in this subsection to test protocols on full model. This is not required as the responses have been saved.

In [ ]:
# Get dictionary with Feature objects for each protocol
nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)

full_model = StnFullModel(name='StnGillies')

# Get features used in optimization
proto_feats = make_opt_features()

In [ ]:
# Run each protocol and get its responses
for proto, feats in proto_feats.iteritems():
    print("Running protocol '{}'".format(proto))

    # Run protocol with full cell model
    responses = proto.run(
                    cell_model=full_model, 
                    param_values={},
                    sim=nrnsim,
                    isolate=False)
    
    plot_responses(responses)

    # Use response to calculate target value for each features
    for feat_name, feat_data in feats.iteritems():

        # Calculate feature value from full model response
        efel_feature, weight = feat_data
        target_value = efel_feature.calculate_feature(responses)

        # Now we can set the target value
        efel_feature.exp_mean = target_value
        efel_feature.exp_std = 0.0

### Create Features, weights, objective function

In [ ]:
# Get dictionary with Feature objects for each protocol
protos_feats = get_features_targets(saved_responses=PROTO_RESPONSES_FILE)

In [ ]:
# Collect characteristic features for all protocols used in evaluation
all_opt_features = []
all_opt_weights = []
for proto, featdict in protos_feats.iteritems():
    feats, weights = zip(*featdict.values())
    all_opt_features.extend(feats)
    all_opt_weights.extend(weights)

# Make final objective function based on selected set of features
total_objective = ephys.objectives.WeightedSumObjective(
            name = 'optimize_all',
            features = all_opt_features,
            weights = all_opt_weights)

## Evaluators & Optimization

In [ ]:
# Create reduced model
red_model = StnReducedModel(
                name		= 'StnFolded',
                fold_method	= 'marasco',
                num_passes	= 7,
                params		= dend_all_params)

nrnsim = ephys.simulators.NrnSimulator(dt=0.025, cvode_active=False)

# Make evaluator to evaluate model using objective calculator
score_calc = ephys.objectivescalculators.ObjectivesCalculator([total_objective])

eval_protos = {proto.name: proto for proto in protos_feats.keys()}
eval_params = [param.name for param in dend_active_params]

cell_evaluator = ephys.evaluators.CellEvaluator(
                    cell_model			= red_model,
                    param_names			= eval_params,
                    fitness_protocols	= eval_protos,
                    fitness_calculator	= score_calc,
                    sim					= nrnsim)

# Make optimization using the model evaluator
optimisation = bpop.optimisations.DEAPOptimisation(
                    evaluator		= cell_evaluator,
                    offspring_size	= 10)

In [ ]:
# Optimize
final_pop, hall_of_fame, logs, hist = optimisation.run(max_ngen=5)

## Analyze optimization results